## 構成ファイルの読み込み

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using System.IO;
using Microsoft.Extensions.Configuration;

var config = new ConfigurationBuilder()
    .AddJsonFile( Path.Combine(Environment.CurrentDirectory, "appsettings.json"))
    .Build();
var endpoint = config["AOAI_ENDPOINT"];
var modelDeploy = config["MODEL_DEPLOY_NAME"];


Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

## Semantic Kernel および 周辺ライブラリを読み込み

本記事執筆時点（2024 年 11 月 12 日）ではライブラリのアップデートが頻繁なためバージョンを 1.28.0 に固定。
今後のアップデートでは動作しなくなる可能性もあるため注意。

- [nuget: Semantic Kernel](https://www.nuget.org/packages/Microsoft.SemanticKernel)


In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.28.0"
#r "nuget: Azure.Identity"
#r "nuget: Microsoft.Extensions.DependencyInjection"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Console"


Installed Packages Azure.Identity, 1.13.1 Microsoft.Extensions.DependencyInjection, 8.0.1 Microsoft.Extensions.Logging, 8.0.1 Microsoft.Extensions.Logging.Console, 8.0.1 Microsoft.SemanticKernel, 1.28.0

### ネイティブ プラグイン を定義

サンプルのため簡単なものを２つだけ定義

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using System.ComponentModel;

public class MyPlugin(ILogger<MyPlugin> logger)
{
    [KernelFunction("get_today")]
    [Description("今日の日付を取得します")]
    [return: Description("今日の日付")]
    public DateTimeOffset GetToday()
    {
        logger.LogInformation("getting today");
        return DateTimeOffset.UtcNow;
    }

    [KernelFunction("get_weather")]
    [Description("指定の日付の天気を回答します。過去日付の場合は履歴を、未来日付の場合は予報をこたえます。")]
    [return: Description("天気")]
    public string GetWeater(DateTimeOffset date)
    {
        logger.LogInformation("getting weather on {date}", date);
        var weather = new string[]{"晴れ", "曇り", "雨", "雪"};
        return weather[new Random().Next(0, weather.Length)];
    }
}

### Kernel にプラグインを読み込んで組み立てる

In [4]:
using Microsoft.Extensions.DependencyInjection;
using Azure.Identity;

// Azure OpenAI
var builder = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(modelDeploy, endpoint, new DefaultAzureCredential());

// ログを標準出力に
builder.Services.AddLogging( lb => {
    //lb.AddFilter("Microsoft.SemanticKernel", LogLevel.Trace);
    lb.AddSimpleConsole(opt => { 
        opt.TimestampFormat = "[hh:mm:ss.fff] "; 
        opt.SingleLine = true;});
});

// プラグインの組み込み
builder.Plugins.AddFromType<MyPlugin>("myplugin");

### まずは直接呼び出すパターン

実用することはあまりないと思うが動作確認として

In [5]:
var kernel = builder.Build();

// 現在時刻を取得するプラグイン
var ret1 = await kernel.InvokeAsync<DateTimeOffset>("myplugin", "get_today");
ret1.Display();

// 天気を取得するプラグインを呼び出す
var args = new KernelArguments(){
    {"date", DateTimeOffset.UtcNow.AddDays(1)}
};
var ret2 = await kernel.InvokeAsync<string>("myplugin", "get_weather", args);
ret2.Display();


[12:24:27.858] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:24:27.861] info: Submission#2.MyPlugin[0] getting today
[12:24:27.862] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:24:27.866] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 0.003047s


2024-11-12 03:24:27Z

[12:24:27.870] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather invoking.
[12:24:27.871] info: Submission#2.MyPlugin[0] getting weather on 11/13/2024 03:24:27 +00:00
[12:24:27.871] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather succeeded.
[12:24:27.871] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather completed. Duration: 0.0014962s


雨

### テンプレートエンジンを使用して呼び出す

In [6]:
var kernel = builder.Build();

var promptText = """
小学生らしい文体で指定された日付の日記を書いてください。

今日の日付 : {{myplugin.get_today}}
""";

var ret3 = await kernel.InvokePromptAsync(promptText);
ret3.Display();

[12:24:28.067] info: Microsoft.SemanticKernel.KernelFunction[0] Function (null)-InvokePromptAsync_ad019231fb044df2b69adb14fc9cb8e5 invoking.
[12:24:28.081] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:24:28.081] info: Submission#2.MyPlugin[0] getting today
[12:24:28.081] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:24:28.081] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 0.0001757s
[12:25:00.331] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 46. Completion tokens: 386. Total tokens: 432.
[12:25:00.338] info: Microsoft.SemanticKernel.KernelFunction[0] Function (null)-InvokePromptAsync_ad019231fb044df2b69adb14fc9cb8e5 succeeded.
[12:25:00.338] info: Microsoft.SemanticKernel.KernelFunction[0] Function (null)-InvokePromptAsync_ad019231fb044df2b69adb14fc9cb8e5 completed. Duration: 32.270877

2024年11月12日　火曜日\n\nこんにちは！今日はとっても楽しい一日だったよ！\n\nまず、朝起きたらお天気がよくて気持ちよかったんだ。朝ごはんはママが焼いてくれたふわふわのパンケーキを食べたよ。バターとメープルシロップをたくさんかけて、美味しかったなぁ。\n\n学校では、今日は図工の授業が楽しかった！みんなで紙粘土を使って自分の好きな動物を作ったんだ。ぼくはうさぎを作ったよ。耳がちょっと大きくなっちゃったけど、先生が「可愛いね！」って褒めてくれて嬉しかった。\n\nお昼休みに、友だちのみんなと校庭でドッジボールをしたんだ。今日はたくさん当てれて、なんだかヒーローになった気分だった！みん... Function InvokePromptAsync_ad019231fb044df2b69adb14fc9cb8e5 Name InvokePromptAsync_ad019231fb044df2b69adb14fc9cb8e5 PluginName <null> Description Generic function, unknown purpose Metadata Microsoft.SemanticKernel.KernelFunctionMetadata Name InvokePromptAsync_ad019231fb044df2b69adb14fc9cb8e5 PluginName <null> Description Generic function, unknown purpose Parameters (empty) ReturnParameter Microsoft.SemanticKernel.KernelReturnParameterMetadata Description ParameterType <null> Schema <null> AdditionalProperties (empty) ExecutionSettings (empty) Metadata key type value Id System.String chatcmpl-ASbo2fwtprzHOiJkEgdYx67D73quN CreatedAt System.DateTimeOffset 2024-11-12 03:24:34Z SystemFingerprint System.String fp_d54531d9eb Usage OpenAI.Chat.ChatTokenUsage OpenAI.Chat.ChatTokenUsage OutputTokenCount 386 InputTokenCount 46 TotalTokenCount 432 OutputTokenDetails <null> Refusal <null> <null> FinishReason System.String Stop ContentTokenLogProbabilities OpenAI.ChangeTrackingList<OpenAI.Chat.ChatTokenLogProbabilityDetails> Culture Parent Parent Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mo

### Function Calling を使用してプラグイン呼び出しの必要性を判定させる

In [7]:
#pragma warning disable SKEXP0001

// プラグインの自動判定
var execSetting = new PromptExecutionSettings(){
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

#### Chat Completion を使う

In [ ]:
using Microsoft.SemanticKernel.ChatCompletion;

// チャット履歴を作成
var history = new ChatHistory();
history.AddSystemMessage("あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。");
history.AddUserMessage("クリスマスまであと何日？");

// チャット補完サービスを利用
var kernel = builder.Build();
var ccsvc = kernel.GetRequiredService<IChatCompletionService>();
var ret5 = await ccsvc.GetChatMessageContentAsync(history, execSetting, kernel);
history.Add(ret5);
history.Display();

// チャット履歴をダンプする(再利用するためにデリゲートにしておく)
var dumpChatHistory = (ChatHistory history) => {
    #pragma warning disable SKEXP0001
    foreach(var msg in history)
    {
        Console.WriteLine($"{msg.Role} : {msg.Content}");
        var functionCalling = FunctionCallContent.GetFunctionCalls(msg);
        if(functionCalling.Any())
        {
            foreach(var fc in functionCalling)
            {
                var funcArgs = fc.Arguments.Count() == 0 ? "" : fc.Arguments.Select(fa => fa.Value).Aggregate((x, y) => $"{x}, {y}");
                Console.WriteLine($"    function_calling: {fc.FunctionName} {funcArgs}");
            }
        }
    }
};

dumpChatHistory(history);


[12:34:17.591] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 127. Completion tokens: 12. Total tokens: 139.
[12:34:17.592] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:34:17.592] info: Submission#2.MyPlugin[0] getting today
[12:34:17.592] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:34:17.592] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 3.87E-05s
[12:34:18.689] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 170. Completion tokens: 26. Total tokens: 196.


[ あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。, クリスマスまであと何日？, , "2024-11-12T03:34:17.59203+00:00", 今日は2024年11月12日なので、クリスマス（12月25日）まであと43日です。 ] Count 5 (values) index type value 0 Microsoft.SemanticKernel.ChatMessageContent あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 AuthorName <null> Role system Label system Content あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 Items [ あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 ] Count 1 (values) index value 0 あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 Text あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 MimeType <null> InnerContent <null> ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 Source <null> MimeType <null> InnerContent <null> ModelId <null> Metadata <null> 1 Microsoft.SemanticKernel.ChatMessageContent クリスマスまであと何日？ AuthorName <null> Role user Label user Content クリスマスまであと何日？ Items [ クリスマスまであと何日？ ] Count 1 (values) index value 0 クリスマスまであと何日？ Text クリスマスまであと何日？ Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 MimeType <null> InnerContent <null> ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 Source <null> MimeType <null> InnerContent <null> ModelId <null> Metadata <null> 2 Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIChatMessageContent ToolCalls index value 0 OpenAI.Chat.ChatToolCall Kind Function FunctionName myplugin-get_today FunctionArguments {} Id call_W7CpVEwIhZc0fXXvyJhSBQCA AuthorName <null> Role Assistant Label Assistant Content <null> Items [ Microsoft.SemanticKernel.FunctionCallContent ] Count 1 (values) index value 0 Microsoft.SemanticKernel.FunctionCallContent Id call_W7CpVEwIhZc0fXXvyJhSBQCA PluginName myplugin FunctionName get_today Arguments (empty) Exception <null> MimeType <null> InnerContent OpenAI.Chat.ChatToolCall Kind Function FunctionName myplugin-get_today FunctionArguments {} Id call_W7CpVEwIhZc0fXXvyJhSBQCA ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True I

system : あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。
user : クリスマスまであと何日？
Assistant : 
    function_calling: get_today 
tool : "2024-11-12T03:34:17.59203+00:00"
Assistant : 今日は2024年11月12日なので、クリスマス（12月25日）まであと43日です。


#### 複数回のプラグイン呼び出し

In [16]:
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0001

// チャット履歴を作成
var history = new ChatHistory();
history.AddSystemMessage("あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。");
history.AddUserMessage("明日の天気を教えて。明日の日付と服装の注意も教えて。");

// チャット補完サービスを利用
var kernel = builder.Build();
var ccsvc = kernel.GetRequiredService<IChatCompletionService>();
var ret5 = await ccsvc.GetChatMessageContentAsync(history, execSetting, kernel);
history.Add(ret5);
history.Display();
dumpChatHistory(history);


[12:35:31.105] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 140. Completion tokens: 12. Total tokens: 152.
[12:35:31.106] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:35:31.106] info: Submission#2.MyPlugin[0] getting today
[12:35:31.106] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:35:31.106] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 6.39E-05s
[12:35:39.273] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 184. Completion tokens: 31. Total tokens: 215.
[12:35:39.273] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather invoking.
[12:35:39.273] info: Submission#2.MyPlugin[0] getting weather on 11/13/2024 00:00:00 +00:00
[12:35:39.273] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-

[ あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。, 明日の天気を教えて。明日の日付と服装の注意も教えて。, , "2024-11-12T03:35:31.1062166+00:00", , 雨, 明日、2024年11月13日の天気は「雨」です。雨が予想されているので、外出する際は傘を持って行くことをおすすめします。また、濡れても大丈夫な靴やレインコートなどを着用するのが良いでしょう。温度変化にも気を付けて、薄手の防寒着もあると安心です。 ] Count 7 (values) index type value 0 Microsoft.SemanticKernel.ChatMessageContent あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 AuthorName <null> Role system Label system Content あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 Items [ あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 ] Count 1 (values) index value 0 あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 Text あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。 Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 MimeType <null> InnerContent <null> ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 Source <null> MimeType <null> InnerContent <null> ModelId <null> Metadata <null> 1 Microsoft.SemanticKernel.ChatMessageContent 明日の天気を教えて。明日の日付と服装の注意も教えて。 AuthorName <null> Role user Label user Content 明日の天気を教えて。明日の日付と服装の注意も教えて。 Items [ 明日の天気を教えて。明日の日付と服装の注意も教えて。 ] Count 1 (values) index value 0 明日の天気を教えて。明日の日付と服装の注意も教えて。 Text 明日の天気を教えて。明日の日付と服装の注意も教えて。 Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 MimeType <null> InnerContent <null> ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 Source <null> MimeType <null> InnerContent <null> ModelId <null> Metadata <null> 2 Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIChatMessageContent ToolCalls index value 0 OpenAI.Chat.ChatToolCall Kind Function FunctionName myplugin-get_today FunctionArguments {} Id call_WAwMiaWYRI2q62ol7czwG6ji AuthorName <null> Role Assistant Label Assistant Content <null> Items [ Microsoft.SemanticKernel.FunctionCallContent ] Count 1 (values) index value 0 Microsoft.SemanticKernel.FunctionCallContent Id call_WAwMiaWYRI2q62ol7czwG6ji PluginName myplugin FunctionName get_today Arguments (empty) Exception <null> MimeType <null> InnerContent OpenAI.Chat.ChatToolCall Kind Function FunctionName myplugin-get_today FunctionArguments {} Id call_WAwMiaWYRI2q62ol7czwG6ji ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8Enco

system : あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。
user : 明日の天気を教えて。明日の日付と服装の注意も教えて。
Assistant : 
    function_calling: get_today 
tool : "2024-11-12T03:35:31.1062166+00:00"
Assistant : 
    function_calling: get_weather 2024-11-13T00:00:00+00:00
tool : 雨
Assistant : 明日、2024年11月13日の天気は「雨」です。雨が予想されているので、外出する際は傘を持って行くことをおすすめします。また、濡れても大丈夫な靴やレインコートなどを着用するのが良いでしょう。温度変化にも気を付けて、薄手の防寒着もあると安心です。


In [15]:
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0001

// チャット履歴を作成
var history = new ChatHistory();
history.AddSystemMessage("あなたはユーザー補助エージェントです。ユーザーの質問に答えてください。");
history.AddUserMessage("過去10年でクリスマスに雪が降った年を教えて");

// チャット補完サービスを利用
var kernel = builder.Build();
var ccsvc = kernel.GetRequiredService<IChatCompletionService>();
var ret6 = await ccsvc.GetChatMessageContentAsync(history, execSetting, kernel);
history.Add(ret6);

// Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit = 30;
// history.Display();
dumpChatHistory(history);

[12:34:43.648] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 136. Completion tokens: 12. Total tokens: 148.
[12:34:43.649] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today invoking.
[12:34:43.649] info: Submission#2.MyPlugin[0] getting today
[12:34:43.649] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today succeeded.
[12:34:43.649] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_today completed. Duration: 1.78E-05s
[12:35:13.407] info: Microsoft.SemanticKernel.Connectors.AzureOpenAI.AzureOpenAIChatCompletionService[0] Prompt tokens: 180. Completion tokens: 226. Total tokens: 406.
[12:35:13.408] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin-get_weather invoking.
[12:35:13.408] info: Submission#2.MyPlugin[0] getting weather on 12/25/2013 00:00:00 +09:00
[12:35:13.408] info: Microsoft.SemanticKernel.KernelFunction[0] Function myplugin